# Imports

In [ ]:
import cv2
import numpy as np
import cv2 as cv
import glob
import time
from tqdm import tqdm
from copy import deepcopy
import scipy.optimize as sopt
from klampt.math import se3,so3
from itertools import combinations
import pickle
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
import open3d as o3d
import pandas as pd
from create_point_cloud import load_whole_point_cloud, load_point_cloud,get_masked_point_cloud
import scipy
import torch
# Copyright (c) OpenMMLab. All rights reserved.
from collections import deque
from queue import Queue
from threading import Event, Lock, Thread
from sklearn.metrics import mean_squared_error


import cv2

from mmpose.apis import (get_track_id, inference_top_down_pose_model,
                         init_pose_model, vis_pose_result)
from mmpose.core import apply_bugeye_effect, apply_sunglasses_effect
from mmpose.utils import StopWatch

try:
    from mmdet.apis import inference_detector, init_detector
    has_mmdet = True
except (ImportError, ModuleNotFoundError):
    has_mmdet = False

try:
    import psutil
    psutil_proc = psutil.Process()
except (ImportError, ModuleNotFoundError):
    psutil_proc = None
from torch import nn
    
from utils import get_proper_image_paths, strip_dataset_parent_folder,get_aligned_dataset,find_hand_center,find_point_position

# Preprocessing the datasets:

In [ ]:
raw_df = pd.read_pickle('./first_ground_truth_shaoxiong_no_sword3.pkl')
clean_df = get_proper_image_paths(raw_df,'/home/motion/data/ECE598/our_dataset')
data = []
mask = np.zeros(133).astype(bool)
mask[:] = False
mask[5:23] = True
mask[91] = True
mask[112] = True
# mask[:] = True
hm = []
for i in tqdm(clean_df.index):  
    try:
        start_time = time.time()
        res,outputs = inference_top_down_pose_model(
                pose_model,
                clean_df.cam_torso_color[i],
                format='xyxy',
                dataset=dataset_name,return_heatmap = False)
        print('Estimating the kp took {}'.format(time.time()-start_time))
        start_time = time.time()
        kp = res[0]['keypoints'][mask,:]
        pcd = load_whole_point_cloud(clean_df.loc[i,'cam_torso_color'],clean_df.loc[i,'cam_torso_depth'],'realsense_torso')
        kp_mean_positions = find_point_position(pcd,kp)
        print('Point Clouds extraction took: {}'.format(time.time()-start_time))
    except Exception as e:
        print(e)
        pass
    hm.append(kp_mean_positions)
        
clean_df['pre_processed_torso_features'] = hm
clean_df.to_pickle('shaoxiong_no_sword_pre_processed3.pkl')

# Motion Forecasting experiments

In [ ]:
MMPOSE_DIR = '/home/motion/Joao/classes/hri_kdc/final_project/mmpose'
det_config = '{}/demo/mmdetection_cfg/ssdlite_mobilenetv2_scratch_600e_coco.py'.format(MMPOSE_DIR)
det_checkpoint = 'https://download.openmmlab.com/mmdetection/v2.0/ssd/ssdlite_mobilenetv2_scratch_600e_coco/ssdlite_mobilenetv2_scratch_600e_coco_20210629_110627-974d9307.pth'
device = 'cuda:0'
enable_human_pose = 1
human_pose_config = '{}/configs/wholebody/2d_kpt_sview_rgb_img/topdown_heatmap/coco-wholebody/vipnas_res50_coco_wholebody_256x192_dark.py'.format(MMPOSE_DIR)
human_pose_checkpoint = 'https://download.openmmlab.com/mmpose/top_down/vipnas/vipnas_res50_wholebody_256x192_dark-67c0ce35_20211112.pth'
human_det_ids = [1]
buffer_size = 1
display_delay = 0
assert has_mmdet, 'Please install mmdet to run the demo.'
assert det_config is not None
assert det_checkpoint is not None

# build detection model
det_model = init_detector(
    det_config, det_checkpoint, device=device.lower())

# build pose models
pose_model_list = []
if enable_human_pose:
    pose_model = init_pose_model(
        human_pose_config,
        human_pose_checkpoint,
        device=device.lower())
    model_info = {
        'name': 'HumanPose',
        'model': pose_model,
        'cat_ids': human_det_ids,
        'bbox_color': (148, 139, 255),
    }
    pose_model_list.append(model_info)


# store pose history for pose tracking
pose_history_list = []

# for _ in range(len(pose_model_list)):
#     pose_history_list.append({'pose_results_last': [], 'next_id': 0})
    
# datasets_dir = '/home/motion/data/ECE598/our_dataset'
# scenes = sorted(glob(datasets_dir + '/*'))

# dataset_folder = scenes[0]

# clean_df =  get_aligned_dataset(dataset_folder,master_camera = 'cam_torso_depth')

In [ ]:
# frame = cv2.imread(clean_df.loc[0,'cam_torso_color'], cv2.IMREAD_COLOR)
dataset_name = pose_model.cfg.data['test']['type']

# res,outputs = inference_top_down_pose_model(
#             pose_model,
#             frame,
#             format='xyxy',
#             dataset=dataset_name,return_heatmap = True)
# heatmap = outputs[0]['heatmap']

In [ ]:
# %%time
# points = get_masked_point_cloud(clean_df.loc[0,'cam_torso_depth'],'realsense_torso')
# points.flatten().shape

In [ ]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [ ]:
class Hands_Dataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, master_df,pose_model,history_length = 5,pred_horizon = 3):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
        """
        self.master_df = master_df
        self.history_length = history_length
        self.pred_horizon = pred_horizon
        self.max_idx = self.master_df.shape[0] - 1
        mask = np.zeros(133)
        mask[:] = False
        mask[5:24] = True
        mask[91:] = True
        self.pose_model = pose_model
        self.mask = mask
        feature_lenght = 10800 + 63
    def __len__(self):
        return self.master_df.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        imgs_idx = np.arange(idx,idx-self.history_length,-1)
        imgs_idx = np.clip(imgs_idx,0,self.max_idx)
        gt_idx = np.clip([idx+self.pred_horizon],0,self.max_idx)[0]
        features = []
        for img_idx in imgs_idx:
#             color = cv2.imread(self.master_df.loc[img_idx,'cam_torso_color'], cv2.IMREAD_COLOR)
            feature = np.array(self.master_df.loc[img_idx,'pre_processed_torso_features'])
            features.append(feature.flatten())
#         colors = np.array(colors)
        features = np.array(features)
        gt_left = self.master_df.loc[gt_idx,'gt_left_hand']
        gt_right = self.master_df.loc[gt_idx,'gt_right_hand']
        gt = np.concatenate((gt_left,gt_right))
#         print(gt)
#         img_name = os.path.join(self.root_dir,
#                                 self.landmarks_frame.iloc[idx, 0])
#         image = io.imread(img_name)
#         landmarks = self.landmarks_frame.iloc[idx, 1:]
#         landmarks = np.array([landmarks])
#         landmarks = landmarks.astype('float').reshape(-1, 2)
        sample = {'data': features, 'gt_pose': gt}


        return sample

In [ ]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers,history_length = 6):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.history_length = history_length
        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True,nonlinearity = 'relu')   
        # Fully connected layert
        self.fc = nn.Linear(hidden_dim, output_size)
        self.device = torch.device('cuda:0')
    def forward(self, x):
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)
        hidden.to(self.device)
        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x.to(self.device), hidden)
        self.intermediate = out
        out = out[:,-1,:]
        # Reshaping the outputs such that it can be fit into the fully connected layer
#         out = out.contiguous().view(batch_size, -1)
        out = self.fc(out)
        
        return out, hidden        
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return hidden

class LSTMModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers,history_length = 6):
        super(LSTMModel, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.history_length = history_length
        #Defining the layers
        # RNN Layer
        self.rnn = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layert
        self.fc = nn.Linear(hidden_dim, output_size)
        self.device = torch.device('cuda:0')
    def forward(self, x):
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        hidden,c = self.init_hidden(batch_size)
        hidden.to(self.device)
        c.to(self.device)
        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x.to(self.device), (hidden,c))
        self.intermediate = out
        out = out[:,-1,:]
        # Reshaping the outputs such that it can be fit into the fully connected layer
#         out = out.contiguous().view(batch_size, -1)
        out = self.fc(out)
        
        return out, hidden        
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        c = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return hidden,c


In [ ]:
pred_horizon = 15
history_length = 10
clean_df = pd.read_pickle('alice_no_sword_pre_processed.pkl')
clean_df2 = pd.read_pickle('alice_no_sword_pre_processed2.pkl')
clean_df = pd.concat([clean_df,clean_df2])
clean_df = clean_df.reset_index(drop = True)
dataset = Hands_Dataset(clean_df,pose_model,pred_horizon = pred_horizon,history_length = history_length)
test_df = pd.read_pickle('alice_no_sword_pre_processed3.pkl')
test_dataset = Hands_Dataset(test_df,pose_model,pred_horizon = pred_horizon,history_length = history_length)

In [ ]:
train_dataloader = DataLoader(dataset, batch_size=16, shuffle=False,num_workers = 4,prefetch_factor= 8,persistent_workers=  True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False,num_workers = 4,prefetch_factor= 8,persistent_workers=  True)
device = torch.device('cuda:0')
# our_model = Model(80,6,250,5,history_length = history_length)
our_model = LSTMModel(80,6,250,1,history_length = history_length)

our_model.to(device)
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(our_model.parameters(), lr=0.0005)
losses = []
for epoch in tqdm(range(1000)):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        x = data['data'].float()
        x.to(device)
        gt_pose = data['gt_pose'].float().to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs,hidden = our_model(x)
        loss = criterion(outputs, gt_pose)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
#         if i % 10 == 9:    # print every 2000 mini-batches
#             print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 10:.3f}')
#             losses.append(running_loss/10)

#             running_loss = 0.0
    losses.append(running_loss/dataset.__len__())

print('Finished Training')

In [ ]:
l = np.array(losses)
plt.scatter(x = range(l.shape[0]),y= l)
plt.xlabel('Batch')
plt.ylabel('MSE Loss')
plt.title('Training loss for horizon = 0.1s - single episode')
plt.savefig('./training_loss_relu.png',bbox_inches = 'tight')

In [ ]:
our_model.eval()
final_loss = 0
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False,num_workers = 1,prefetch_factor= 4,persistent_workers=  True)
gts = []
preds = []
for data in tqdm(test_dataloader):
    x = data['data'].float()
    x.to(device)
    gt_pose = data['gt_pose'].float().to(device)
    with torch.no_grad():
#         start_time = time.start()
        # zero the parameter gradients
        # forward + backward + optimize
        outputs,hidden = our_model(x)
        preds.append(outputs.cpu().numpy())
        gts.append(gt_pose.cpu().numpy())
        loss = criterion(outputs, gt_pose)
        final_loss += loss



In [ ]:
# help(mean_squared_error)

In [ ]:
mean_squared_error(np.array(gts).reshape(-1,6),np.array(preds).reshape(-1,6),squared = False)

RNN: (80,6,250,1,horizon_length = 10)
MSE for 0.5 seconds horizon: 0.3864m
MSE for 5/30 seconds horizon: 0.3181
MSE for 0.1 seconds horizon: 0.2897

RNN: (80,6,250,5,history_length = 10)
MSE for 0.5 seconds horizon: 0.3337
MSE for 5/30 seconds horizon: 0.3180
MSE for 0.1 seconds horizon: 0.2831

LSTM:(80,6,250,5,history_length = 10)
MSE for 0.5 seconds horizon: 0.3372
MSE for 5/30 seconds horizon: 0.3124
MSE for 0.1 seconds horizon: 0.2901

LSTM:(80,6,250,1,history_length = 10)
MSE for 0.5 seconds horizon: 0.31
MSE for 5/30 seconds horizon: 0.2994
MSE for 0.1 seconds horizon: 0.2649

Baseline:
MSE for 0.5 seconds horizon: 0.3812
MSE for 5/30 seconds horizon:0.3220
MSE for 0.1 seconds horizon: 0.3097

In [ ]:
import pandas as pd

performance_dict = {'network':['Baseline','Baseline','Baseline','RNN_1','RNN_1','RNN_1','RNN_5','RNN_5','RNN_5','LSTM_1','LSTM_1','LSTM_1','LSTM_5','LSTM_5','LSTM_5'],
                   'Horizon (s)':[15/30,np.round(5/30,3),3/30,15/30,np.round(5/30,3),3/30,15/30,np.round(5/30,3),3/30,15/30,np.round(5/30,3),3/30,15/30,np.round(5/30,3),3/30],
                   'RMSE (m)':[0.3812,0.3220,0.3097,0.3864,0.3181,0.2897,0.3337,0.3180,0.2831,0.3372,0.3124,0.2901,0.31,0.2994,0.2649]}
performance_df = pd.DataFrame(performance_dict)

import seaborn as sns
sns.set_theme()


sns.catplot(x='Horizon (s)', y='RMSE (m)', hue='network', data=performance_df, kind='bar')

plt.savefig('./Test_Error_LSTM_RNN.pdf',bbox_inches = 'tight')

# non-learning baseline:

In [ ]:
def get_last_valid_value(array,index):
    valid_idxs = np.where(array[:,index,:3] != [0,0,0])
#     print(valid_idxs)
    if(len(valid_idxs[0]) > 0):
        max_valid_idx = valid_idxs[0].max()
#         print(array.shape)
        return True,array[max_valid_idx,index,:3]
    else:
        return False,[0,0,0]

In [ ]:
# our_model.eval()
final_loss = 0
pred_horizon = 15
test_df = pd.read_pickle('alice_no_sword_pre_processed3.pkl')
test_dataset = Hands_Dataset(test_df,pose_model,pred_horizon = pred_horizon,history_length = history_length)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False,num_workers = 1,prefetch_factor= 4,persistent_workers=  True)
gts = []
preds = []
last_valid_right = False
last_valid_left = False
valid_left_pose = [0,0,0]
valid_right_pose = [0,0,0]
for data in tqdm(test_dataloader):
    x = data['data'].float().numpy().reshape(-1,20,4)
#     x.to(device)
    gt_pose = data['gt_pose'].float().to(device)
    valid_l,left_pose = get_last_valid_value(x,18)
    valid_r,right_pose = get_last_valid_value(x,19)
    if(valid_l):
        valid_left_pose = left_pose
    if(valid_r):
        valid_right_pose = right_pose
    
    pred = np.concatenate([valid_left_pose,valid_right_pose])
    preds.append(pred)
    with torch.no_grad():
#         start_time = time.start()
        # zero the parameter gradients
        # forward + backward + optimize
#         outputs,hidden = our_model(x)
#         preds.append(outputs.cpu().numpy())
#         break
        gts.append(gt_pose.cpu().numpy())


In [ ]:
mean_squared_error(np.array(gts).reshape(-1,6),np.array(preds).reshape(-1,6),squared = False)

In [ ]:
get_last_valid_value(x,18)